In [12]:
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt
import seaborn as sns

In [13]:
%%time
adata = sc.read('./GSE194122_openproblems_neurips2021_multiome_BMMC_processed.h5ad')
print(adata)

AnnData object with n_obs × n_vars = 69249 × 129921
    obs: 'GEX_pct_counts_mt', 'GEX_n_counts', 'GEX_n_genes', 'GEX_size_factors', 'GEX_phase', 'ATAC_nCount_peaks', 'ATAC_atac_fragments', 'ATAC_reads_in_peaks_frac', 'ATAC_blacklist_fraction', 'ATAC_nucleosome_signal', 'cell_type', 'batch', 'ATAC_pseudotime_order', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker'
    var: 'feature_types', 'gene_id'
    uns: 'ATAC_gene_activity_var_names', 'dataset_id', 'genome', 'organism'
    obsm: 'ATAC_gene_activity', 'ATAC_lsi_full', 'ATAC_lsi_red', 'ATAC_umap', 'GEX_X_pca', 'GEX_X_umap'
    layers: 'counts'
CPU times: user 46.1 s, sys: 4.39 s, total: 50.5 s
Wall time: 50.5 s


In [14]:
adata.obs.head()

,GEX_pct_counts_mt,GEX_n_counts,GEX_n_genes,GEX_size_factors,GEX_phase,ATAC_nCount_peaks,ATAC_atac_fragments,ATAC_reads_in_peaks_frac,ATAC_blacklist_fraction,ATAC_nucleosome_signal,...,VendorLot,DonorID,DonorAge,DonorBMI,DonorBloodType,DonorRace,Ethnicity,DonorGender,QCMeds,DonorSmoker
TAGTTGTCACCCTCAC-1-s1d1,1.061008,1508.0,1022,0.453484,S,4031.0,5400,0.746481,0.003473,0.642468,...,3054455,15078,34,24.8,B-,White,HISPANIC OR LATINO,Male,False,Nonsmoker
CTATGGCCATAACGGG-1-s1d1,0.604230,1655.0,1081,0.455631,G2M,8636.0,19266,0.448251,0.003126,1.220679,...,3054455,15078,34,24.8,B-,White,HISPANIC OR LATINO,Male,False,Nonsmoker
CCGCACACAGGTTAAA-1-s1d1,0.650069,7230.0,3304,2.435348,G2M,4674.0,6177,0.756678,0.001284,0.692573,...,3054455,15078,34,24.8,B-,White,HISPANIC OR LATINO,Male,False,Nonsmoker
TCATTTGGTAATGGAA-1-s1d1,0.812274,1108.0,793,0.347226,G2M,2803.0,4019,0.697437,0.000714,0.633838,...,3054455,15078,34,24.8,B-,White,HISPANIC OR LATINO,Male,False,Nonsmoker
ACCACATAGGTGTCCA-1-s1d1,1.674770,1851.0,1219,0.534205,G2M,1790.0,2568,0.697040,0.003352,0.727660,...,3054455,15078,34,24.8,B-,White,HISPANIC OR LATINO,Male,False,Nonsmoker


In [15]:
X = adata.X

In [16]:
labels = adata.obs['cell_type']

In [17]:
from sklearn.neighbors import LocalOutlierFactor
from scipy.sparse import issparse, csr_matrix

def remove_outliers_and_labels(adata, label_key='cell_type', n_top_genes=5000, n_pcs=50, n_neighbors=20, contamination='auto', copy=False):
    if copy:
        adata = adata.copy()
    
    if label_key not in adata.obs:
        raise ValueError(f"Label key '{label_key}' not found in adata.obs")
    
    original_X = adata.X
    labels = adata.obs[label_key].copy()
    
    adata_norm = adata.copy()
    sc.pp.filter_genes(adata_norm, min_cells=10)
    sc.pp.normalize_total(adata_norm, target_sum=1e4)
    sc.pp.log1p(adata_norm)
    
    sc.pp.highly_variable_genes(adata_norm, n_top_genes=n_top_genes, subset=True)
    sc.tl.pca(adata_norm, n_comps=n_pcs, svd_solver='arpack')
    
    lof = LocalOutlierFactor(n_neighbors=n_neighbors, contamination=contamination)
    outliers = lof.fit_predict(adata_norm.obsm['X_pca']) == -1
    
    adata_filtered = adata[~outliers, :].copy()
    
    adata_filtered.X = original_X[~outliers, :]
    if issparse(original_X):
        adata_filtered.X = csr_matrix(adata_filtered.X) if not issparse(adata_filtered.X) else adata_filtered.X
    
    filtered_labels = labels[~outliers]
    
    print(f'Удалено {outliers.sum()} выбросов из {len(outliers)} клеток')
    print(f'Осталось {len(filtered_labels)} меток')
    
    return adata_filtered, filtered_labels

In [18]:
adata, labels = remove_outliers_and_labels(adata, label_key='cell_type', copy=True)

Удалено 2069 выбросов из 69249 клеток
Осталось 67180 меток


In [19]:
X = adata.X

In [20]:
from scipy.sparse import csr_matrix

n, p = X.shape

mean = X.mean(axis=0).A1  

rows, cols = X.nonzero()
data_cen = X.data - mean[cols]
X_cen = csr_matrix((data_cen, (rows, cols)), shape=(n, p))

In [21]:
std = np.sqrt(X_cen.power(2).mean(axis=0)).A1
std[std == 0] = 1

Y_std = data_cen / std[cols]
Y = csr_matrix((Y_std, (rows, cols)), shape=X.shape)

In [22]:
from sklearn.utils.extmath import randomized_svd

n_components = 4000 
random_state = 42   

U, S, Vt = randomized_svd(Y, n_components=n_components, random_state=random_state)
n, p = Y.shape

In [23]:
q = p / n
eigenvalues = (S ** 2) / (n - 1)
lambda_max = (1 + np.sqrt(q))**2

In [24]:
significant_mask = eigenvalues > lambda_max
significant_eigenvalues = eigenvalues[significant_mask]

print(f'Всего вычислено собственных значений: {len(eigenvalues)}')
print(f'Параметр q (p/n): {q:.4f}')
print(f'Теоретическая граница случайного спектра (lambda_max): {lambda_max:.4f}')
print(f'Найдено значимых собственных значений: {len(significant_eigenvalues)}')
print(f'Значимые собственные значения: {significant_eigenvalues}')

Всего вычислено собственных значений: 4000
Параметр q (p/n): 1.9339
Теоретическая граница случайного спектра (lambda_max): 5.7152
Найдено значимых собственных значений: 3022
Значимые собственные значения: [5484.6025     878.05835    692.57074   ...    5.7175364    5.716175
    5.715971 ]


In [25]:
significant_components = Vt[significant_mask] 

Z = Y @ significant_components.T 

In [26]:
np.save('Z.npy', Z)

In [27]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report
from sklearn.multioutput import MultiOutputClassifier
import numpy as np
import pandas as pd

In [28]:
print(Z.shape)
print(labels.shape)

(67180, 3022)
(67180,)


In [29]:
if isinstance(labels, pd.Series):
    labels = labels.values

if len(labels.shape) == 1:
    labels = labels.reshape(-1, 1)

In [30]:
X_train, X_test, y_train, y_test = train_test_split(
    Z, 
    labels, 
    test_size=0.2, 
    random_state=42, 
    stratify=labels if len(np.unique(labels)) > 1 else None
)

In [31]:
rf = RandomForestClassifier(
    n_estimators=100,
    random_state=42,
    n_jobs=-1,
    verbose=0
)

catboost = CatBoostClassifier(
    iterations=100,
    random_state=42,
    verbose=0,
    task_type='CPU'
)

logreg = LogisticRegression(
    max_iter=1000,
    random_state=42,
    n_jobs=-1
)


ensemble = VotingClassifier(
    estimators=[
        ('rf', rf),
        ('catboost', catboost),
        ('logreg', logreg)
    ],
    voting='soft'  
)

In [32]:
model = MultiOutputClassifier(ensemble, n_jobs=-1)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print("Отчет по классификации:")
for i in range(y_test.shape[1]):
    print(f"\n=== Метка {i+1} ===")
    print(classification_report(
        y_test[:, i], 
        y_pred[:, i], 
        zero_division=0,
        digits=3
    ))

print(f"Accuracy: {np.mean(y_pred == y_test):.3f}")

Exception ignored in: <function ResourceTracker.__del__ at 0x795968995f80>
Traceback (most recent call last):
  File "/home/fedor/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/home/fedor/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/home/fedor/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 116, in _stop_locked
ChildProcessError: [Errno 10] No child processes
Exception ignored in: <function ResourceTracker.__del__ at 0x70566678df80>
Traceback (most recent call last):
  File "/home/fedor/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/home/fedor/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/home/fedor/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 116, in _stop_locked
ChildProcessError: [Errno 10] No child processes
Exception ignored in: <function ResourceTracker.__del__ 

Отчет по классификации:

=== Метка 1 ===
                     precision    recall  f1-score   support

               B1 B      0.914     0.848     0.879       374
         CD14+ Mono      0.953     0.977     0.965      2109
         CD16+ Mono      0.933     0.920     0.926       362
   CD4+ T activated      0.683     0.654     0.668      1074
       CD4+ T naive      0.645     0.654     0.649       849
             CD8+ T      0.872     0.919     0.895      2271
       CD8+ T naive      0.648     0.538     0.588       199
       Erythroblast      0.950     0.948     0.949       963
           G/M prog      0.631     0.563     0.595       222
                HSC      0.711     0.711     0.711       201
ID2-hi myeloid prog      1.000     0.471     0.640        17
                ILC      0.752     0.510     0.608       155
         Lymph prog      0.849     0.846     0.847       338
          MK/E prog      0.789     0.671     0.725       167
                 NK      0.917     0.948   

In [33]:
not_significant_mask = eigenvalues <= lambda_max
not_significant_eigenvalues = eigenvalues[not_significant_mask]

In [34]:
test_significant_eigenvalues = np.concatenate([significant_eigenvalues, not_significant_eigenvalues[-20:]])

In [37]:
np.save('eigenvalues.npy', eigenvalues)

In [38]:
np.save('Y.npy', Y)
np.save('Vt.npy', Vt)

In [45]:
labels = pd.DataFrame({'labels': list(labels)})
labels.to_csv('labels.csv', index=False)

In [46]:
np.save('U.npy', Vt)

In [47]:
print(lambda_max)

5.71523555422111


In [49]:
from scipy.sparse import save_npz, load_npz

save_npz('Y.npz', Y)